In [1]:
import numpy as np
import sqlite3
import pandas as pd
import csv
from datetime import datetime, timedelta
import time
import datetime
import requests
from io import StringIO

In [2]:
save_list=np.load("save_list.npy")
now_list=np.load("now.npy")

In [84]:
## Open file   把代號txt  拿出來
fp = open('911.txt', "r")
line = fp.readline()
month_predict=[]
## 用 while 逐行讀取檔案內容，直至檔案結尾
while line:
#    print (line)
    line=line.replace(" ","")
    month_predict.append(line.replace("\n",""))
    line = fp.readline()
fp.close()

In [82]:
#把當月11號的台股資訊 更新
datestr = '20200911'
# 下載股價
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')
# 整理資料，變成表格
df = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
# 整理一些字串：
df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
# 顯示出來
#df.head()

In [85]:
month_predict

['2884',
 '2887',
 '2885',
 '8111',
 '2206',
 '3285',
 '1477',
 '3272',
 '5471',
 '8499',
 '1589',
 '4906',
 '3661',
 '8938',
 '4977',
 '3163',
 '2352']

In [86]:
for i in range(len(month_predict)):
    a=df[pd.to_numeric(df['證券代號'], errors='coerce') == float(month_predict[i])]
    try:
        b=a.values.tolist()[0]
        print(b[0])
    except:
        pass

2884.0
2887.0
2885.0
2206.0
1477.0
5471.0
8499.0
1589.0
4906.0
3661.0
4977.0
2352.0


In [57]:
#抓前一天的台股資訊 更新
datestr = '20200922'
# 下載股價
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')
# 整理資料，變成表格
df = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
# 整理一些字串：
df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
# 顯示出來
#df.head()

In [87]:
a=df[pd.to_numeric(df['證券代號'], errors='coerce') == 2330]
a

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16


In [53]:
a.values.tolist()[0]

[2330.0,
 nan,
 36842639.0,
 22692.0,
 16140184501.0,
 440.0,
 441.0,
 436.0,
 437.0,
 -1.0,
 3.0,
 436.5,
 310.0,
 437.0,
 388.0,
 24.91,
 nan]

In [3]:
df = pd.read_csv('公司/上市.csv')  
a=df["有價證券代號及名稱"].tolist()
df = pd.read_csv('公司/上櫃.csv')  
b=df["有價證券代號及名稱"].tolist()
df = pd.read_csv('公司/興櫃.csv')  
c=df["有價證券代號及名稱"].tolist()



In [4]:
#查詢list in str
#any('1101' in item for item in a)
#查詢list in str
def search_name(name):
    company=[]
    
    for i in a:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in b:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in c:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    return company

In [5]:
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicHistory")
db.commit()
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from technicHistory")
db.commit()

In [6]:
#基本面歷史
pk=1
for i in range(len(save_list)):
    name=search_name(save_list[i][0]).replace("\u3000", " ")
    start_date=save_list[i][1]
    over_date=save_list[i][2]
    buy_price=save_list[i][3]
    sell_price=save_list[i][4]
    return_value=(float(sell_price)-float(buy_price))/float(buy_price)
    return_value= round(return_value*100,2)
    if return_value>0:
        types="+"
    else:
        types="-"

    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicHistory (id,stock_name,start_date,buy_price,over_date,sell_price,return_value,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,name,start_date,buy_price,over_date,sell_price,return_value,types))
    db.commit()
    db.close()
    pk=pk+1

In [7]:
#技術面歷史
pk=1
for i in range(len(save_list)):
    name=search_name(save_list[i][0]).replace("\u3000", " ")
    start_date=save_list[i][1]
    over_date=save_list[i][2]
    buy_price=save_list[i][3]
    sell_price=save_list[i][4]
    return_value=(float(sell_price)-float(buy_price))/float(buy_price)
    return_value= round(return_value*100,2)
    if return_value>0:
        types="+"
    else:
        types="-"

    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicHistory (id,stock_name,start_date,buy_price,over_date,sell_price,return_value,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,name,start_date,buy_price,over_date,sell_price,return_value,types))
    db.commit()
    db.close()
    pk=pk+1

In [8]:
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicCurrent")
db.commit()
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from technicCurrent")
db.commit()

In [10]:
#基本面現在
pk=1
for i in range(len(now_list)):
    final_update="2020-07-14"
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date="2020-07-10"
    start_price=now_list[i][2]
    over_date="2020-08-14"
    current_price=now_list[i][3]
    now_return=now_list[i][1]
    if float(now_return)>0:
        types="+"
    else:
        types="-"
        
        
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1

In [11]:
#基本面現在
pk=1
for i in range(len(now_list)):
    final_update="2020-07-14"
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date="2020-07-10"
    start_price=now_list[i][2]
    over_date="2020-08-14"
    current_price=now_list[i][3]
    now_return=now_list[i][1]
    if float(now_return)>0:
        types="+"
    else:
        types="-"
        
        
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1